In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 26 17:26:10 2020
@author: Rai Kanwar Taimoor
@adapted by Wenhan Yang on Wed May 5 2020
"""

import numpy as np
import pandas as pd
import matplotlib.image as img
from prettytable import PrettyTable
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

def to_lower(arr):
    for i in range(0,len(arr)):
        arr[i] = arr[i].lower()
    return arr
def Result_Table(table_col1, table_col2):
    table = PrettyTable()
    table.add_column("Actual Label", table_col1)
    table.add_column("Predicted Value", table_col2)
    return table
def get_pc(raw_matrix, pca):
    #PCA to get the principle component(a vector)
    pc = pca.fit_transform(raw_matrix)
    explained_variane = pca.explained_variance_ratio_
    return pc, explained_variane
def convertImageToGray(image, cvtr):
    #convert RGB(3-channel)/RGBA(4-channel) pics to 2-channel greyscale
    gray = cvtr.cvtColor(image, cvtr.COLOR_BGR2GRAY)
    return gray
def vectorize_matrix(mat):
    #sraighten the matrix into a column
    col_vector = np.reshape(mat, (len(mat)*len(mat[0]),1))
    return col_vector
def get_pic_matrix(input_data, filepath):
    #initialize a new array to hold vectorized pics
    input_pics = []
    #two new list to record imgs' idx where they fail to open or process
    idx_exception = []
    #transfrom the corresponding pictures into matrices, then vectors.
    #for train_pics in X_train:
    for i in range(0, len(input_data)):
        try:
            image = img.imread(filepath + input_data[i])
            if (len(image.shape) > 2):
                #sklearn can only process 2D array
                #so convert 2D Greyscale into RGB
                image = convertImageToGray(image, img)
            elif (len(image.shape) < 2):
                #record "throw-away" pics' idx
                idx_exception.append(i)
                continue
        except:
            #record "throw-away" pics' idx
            idx_exception.append(i)
            continue
        #img_vector, explained_variane = get_pc(image,pca)
        input_pics.append(vectorize_matrix(image))
    return input_pics, idx_exception

In [2]:
# Initialization
my_data = pd.read_csv('facial_expressions-master/data/legend.csv').values
X_data = my_data[0:2000, 1]
y_data = my_data[0:2000, 2]
y_data = to_lower(y_data)

# Spiliting Data 67-33 ratio as said by sir
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_data,y_data,test_size=0.33,random_state=0)

In [3]:
my_path = "facial_expressions-master/images/"
X_train_pics, idx_train_exception = get_pic_matrix(X_train, my_path)
X_test_pics, idx_test_exception = get_pic_matrix(X_test, my_path)
#throw away unprocessable imgs' idx for y data
y_train = np.delete(y_train,idx_train_exception,axis = 0)
y_test = np.delete(y_test,idx_test_exception,axis = 0)
#convert X_data to ndarray
X_train_pics = np.array(X_train_pics)
X_test_pics = np.array(X_test_pics)
X_train_pics = X_train_pics.reshape((len(X_train_pics),len(X_train_pics[0])))
X_test_pics = X_test_pics.reshape((len(X_test_pics),len(X_test_pics[0])))

658
640


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 5
batch_size = 4
learning_rate = 0.001
classes = ('anger', 'contempt', 'happiness', 'neutral',
           'sadness', 'surprise')

In [5]:
# This block is for loading data. THe author uses existing data package 'CIFAR10' here.
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# CIFAR10: 60000 32x32 color images in 10 classes, with 6000 images per class
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x
    
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [10]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(len(classes)):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Epoch [1/5], Step [2000/12500], Loss: 1.0997
Epoch [1/5], Step [4000/12500], Loss: 1.1197
Epoch [1/5], Step [6000/12500], Loss: 1.0221
Epoch [1/5], Step [8000/12500], Loss: 1.1469
Epoch [1/5], Step [10000/12500], Loss: 1.1395
Epoch [1/5], Step [12000/12500], Loss: 0.4078
Epoch [2/5], Step [2000/12500], Loss: 1.6763
Epoch [2/5], Step [4000/12500], Loss: 0.8741
Epoch [2/5], Step [6000/12500], Loss: 0.7569
Epoch [2/5], Step [8000/12500], Loss: 0.9538
Epoch [2/5], Step [10000/12500], Loss: 0.8128
Epoch [2/5], Step [12000/12500], Loss: 1.0864
Epoch [3/5], Step [2000/12500], Loss: 1.6601
Epoch [3/5], Step [4000/12500], Loss: 1.0568
Epoch [3/5], Step [6000/12500], Loss: 0.4385
Epoch [3/5], Step [8000/12500], Loss: 0.5612
Epoch [3/5], Step [10000/12500], Loss: 0.6724
Epoch [3/5], Step [12000/12500], Loss: 0.4491
Epoch [4/5], Step [2000/12500], Loss: 1.0004
Epoch [4/5], Step [4000/12500], Loss: 0.8073
Epoch [4/5], Step [6000/12500], Loss: 0.9559
Epoch [4/5], Step [8000/12500], Loss: 0.9461
Epoc